In [ ]:
# Data from Copyright © Global Energy Monitor. Distributed under a Creative Commons Attribution 4.0 International License.
# Recommended Citation: “Global Energy Monitor, Global Oil and Gas Extraction Tracker, March 2024 Release”. When the data set is being shared or adapted and our Creative Commons CC BY 4.0 International license applies, please read the license for attribution requirements. Necessary attribution elements are included in our data download files.
# https://globalenergymonitor.org/projects/global-oil-gas-extraction-tracker/

In [ ]:
import pandas as pd
import geojson, json, os, random

class GoogleSheetRow():
    columns = [
        'Enabled', 'Share link identifier', 'Category', 'Name', 'Credits', 'Base layer', 'Custom slider ticks', 'Start date',
        'End date', 'Step', 'URL', 'Scaling', 'Color Scaling', 'Map Type', 'Color', 'External GeoJSON', 'Name Key', 'Show Graph',
        'Graph Title', 'Graph Plot Colors', 'Graph Plots First Visible', 'Graph X-Axis Label', 'Graph Y-Axis Label', 'Graph Y-Axis Min',
        'Graph Y-Axis Max', 'Graph X-Axis Label Interval', 'Legend Content', 'Legend Key', 'Load Data Function', 'Set Data Function', 
        'Number of Levels', 'Number of Attributes', 'Vertex Shader', 'Fragment Shader', 'Draw Function', 'Playback Rate',
        'Master Playback Rate', 'Colormap Src', 'Layer Description', 'Featured Themes', 'Draw Options', 'Set Data Options', 'Extras Options',
        'Draw Order', 'Timeline Type', 'Layer Constraints', 'Mapbox', 'Draw Layer Function'
    ]
    key_values = {}
    def __init__(self, _dict = {}):
        for column in self.columns:
            self.key_values[column] = ''
        for key in _dict.keys():
            if key in self.columns:
                self.key_values[key] = _dict[key]
    def set_column(self, key, value):
        if key in self.columns:
            self.key_values[key] = value
    def print_row(self):
        row = []
        for column in self.columns:
            row.append(self.key_values[column])
        print("\t".join(row))
        


In [ ]:
fname = "../data/Global-Oil-and-Gas-Extraction-Tracker-March-2024.xlsx"
excel = pd.ExcelFile('../data/Global-Oil-and-Gas-Extraction-Tracker-March-2024.xlsx')
excel.sheet_names

In [ ]:
df = pd.read_excel(excel, "Main data")  

In [ ]:
feature_collection = geojson.FeatureCollection(features=[])
max_value = 0
for idx, row in df.iterrows():
    if row['Status'] != 'operating' or pd.isna(row['Production start year']) or pd.isna(row['Latitude']):
        continue
    point = geojson.Point((row['Longitude'], row['Latitude']))
    # Why add a random month to start?
    #   We could be exact and just add the start year, but the visualization ends feeling chunky
    #   Adding a somewhat staggered random start to each inidvidual location makes it seem nicer
    #   Just opinions though. Do w/e you want and pick what you like best
    random_month = random.randrange(1,12)
    random_pad_1 = str(random_month).zfill(2) + '01'    
    random_pad_2 = str(random_month+1).zfill(2) + '01'    

    dates = [str(int(row['Production start year']))+random_pad_1, str(int(row['Production start year']))+random_pad_2, '20240101']
    values = [0,100,100]
    properties = {
        'status': 'operating',
        'timeseries': {
            'dates': dates,
            'values': values
        }
        
    }
    feature = geojson.Feature(geometry=point, properties=properties)
    feature_collection['features'].append(feature)


In [ ]:
oname = 'data/making_a_marker_map_example_1.geojson'
os.makedirs(os.path.dirname(oname), exist_ok=True)
with open(oname, 'w') as f:
    json.dump(feature_collection, f)


In [ ]:
# Copy the data somewhere www accessible
#!scp data/making_a_marker_map_example_1.geojson timemachine2:/t/earthtime.org/app/data/making-earthtime-maps
!scp data/making_a_marker_map_example_1.geojson example_machine:/example/path/

In [ ]:
# Please try to always add a layer description with minimally a brief explanation of what the visualization shows 
#  and where the data came from.
layer_description = "<h2>Oil and Gas Extraction Sites</h2><p>Sourced from <a href=\"https://globalenergymonitor.org/projects/global-oil-gas-extraction-tracker/\">https://globalenergymonitor.org/projects/global-oil-gas-extraction-tracker/</a></p><p>Displaying operating oil and gas extraction sites.</p>"
# Share link identifiers should be unique. Would be great if a DB just handled this... 
share_link_identifer = 'making_a_marker_map_example_1_' + str(random.getrandbits(16))
category = 'Making EarthTime Maps'
name = 'Oil and Gas Extraction Sites'
# This should be wherever yoou uploaded your data to
url = 'https://tiles.earthtime.org/making-earthtime-maps/making_a_marker_map_example_1.geojson'
layer_args = {
    'Enabled': 'TRUE',
    'Share link identifier': share_link_identifer,
    'Category': category, 
    'Name': name, 
    'Credits': 'Global Energy Monitor', 
    'Base layer': 'mb_openplanetv1', 
    'Start date': '1900',
    'End date': '2024', 
    'URL': url, 
    'Scaling': f'd3.scaleSqrt().domain([0,100]).range([0,10]).clamp(true)', 
    'Layer Description': layer_description,
    'Map Type': 'marker',
    'Load Data Function': 'WebGLVectorTile2.prototype._loadGeojsonData',
    'Set Data Function': 'WebGLVectorTile2.prototype._setMarkerData',
    'Number of Attributes': '14',
    'Vertex Shader': 'WebGLVectorTile2.animatedMarkerVertexShader',
    'Fragment Shader': 'WebGLVectorTile2.markerFragmentShader',
    'Draw Function': 'WebGLVectorTile2.prototype._drawMarker', 
    'Draw Options': '{"pointSize": 10, "linewidth": 2, "mode": "disc"}',
    'Set Data Options': '{ "sizeKey": "timeseries", "fills": [[250, 242, 232, 0.55]], "strokes": [[250, 242, 232, 0.75]]}',
    'Draw Order': '700',
}
row = GoogleSheetRow(layer_args)

In [ ]:
# C&P the output of this into a sheet. Would be cool if this just like happened...
row.print_row()